In [ ]:
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *

import cv2, os
import numpy as np
from scipy.ndimage import zoom
from os.path import *
# import PIL
# from PIL import Image
import matplotlib.pyplot as plt
import glob, json

contentPath=os.getcwd()
planetsPath=join(contentPath, 'planets')
if not exists(planetsPath):
    tff.extractTarfiles('OriginalPlanets.tar.gz')

imagePath=join(contentPath, '3b7d7d8a64.jpg')
savePath=join(contentPath, 'tmp')
if not exists(savePath):
    os.makedirs(savePath)
    
initialGlobList:list
with open("initialGlobList.json", 'r') as f:
    initialGlobList = json.load(f)
    
def listNewFiles(initial=initialGlobList, delete=False):
    currentFilesGlob=glob.glob('**', recursive=False)
    print(f'New files...')
    if len(initialGlobList) == len(currentFilesGlob):
        print(f'{C.BIRed}No new files.\nNothing to see here.')
    for fil in currentFilesGlob:
        if not fil in initial:
            if isdir(fil):
                print(f'{C.BIBlue}{fil}')
                if delete:
                    shutil.rmtree(fil)
            elif isfile(fil):
                print(f'{C.ColorOff}{fil}')
                if delete:
                    os.remove(fil)
listNewFiles()

In [ ]:
def cropSquare(img, size, silent=True):
    ''' '''
    # print(f'initial shape: {img.shape}')
    h, w = img.shape[:2]
    min_size = np.amin([h,w])
    # print(f'min_size: {min_size}')

    # Centralize and crop
    crop_img = img[int(h/2-min_size/2):int(h/2+min_size/2), 
                   int(w/2-min_size/2):int(w/2+min_size/2)]
    resized = cv2.resize(crop_img, (size, size), 
                         interpolation=cv2.INTER_AREA)
    resized = cv2.resize(resized, (224, 224))
    if not silent:
        cvu.plotShowTwoImages(img, resized,
                              title1=img.shape, title2=resized.shape)
    return resized

imagePath='/home/jovyan/planets/BC503786-D500-4389-A1AC-2ABC8DA1D75F.jpeg'
img=cv2.imread(imagePath, cv2.IMREAD_COLOR)
img=cropSquare(img, 224, silent=False)

img = img[:,:,::-1]
fig, ax = plt.subplots(1, 2)
ax[0].imshow(img)
ax[1].imshow(img)

In [ ]:
# Resizes a image and maintains aspect ratio
def resize_maintain_aspect_ratio(path, width=224, height=224, silent=True):
    ''' '''
    image=cv2.imread(path, cv2.IMREAD_COLOR)
    image=np.copy(image)
    # print(type(image))
    # Grab the image size and initialize dimensions
    dim = None
    (h, w) = image.shape[:2]

    # Return original image if no need to resize
    if width is None and height is None:
        return image

    # We are resizing height if width is none
    if width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # We are resizing width if height is none
    else:
        # Calculate the ratio of the 0idth and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Return the resized image
    resizedImage = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    
    if resizedImage.shape != (224, 224, 3):
        resizedImage=cropSquare(resizedImage, 224)
        print(resizedImage.shape)
    
    if not silent:
        print(image.shape, 'resized to ->', resizedImage.shape)
        cvu.plotShowTwoImages(image, resizedImage,
                              title1=image.shape,
                              title2=resizedImage.shape)
    return resizedImage

imagePath='/home/jovyan/planets/BC503786-D500-4389-A1AC-2ABC8DA1D75F.jpeg'
resizedImage=resize_maintain_aspect_ratio(path=imagePath,
                                  width=224, height=224,
                                  silent=False)

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA, silent=True):
    ''' '''
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]
    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image
    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))
    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)
    # return the resized image
    return resized

imagePath='/home/jovyan/planets/BC503786-D500-4389-A1AC-2ABC8DA1D75F.jpeg'
img=cv2.imread(imagePath, cv2.IMREAD_COLOR)
resizedImage=image_resize(img, 224, silent=False)
print(resizedImage.shape)

img = img[:,:,::-1]
resizedImage=resizedImage[:,:,::-1]
fig, ax = plt.subplots(1, 2)
ax[0].imshow(img)
ax[1].imshow(resizedImage)

In [ ]:
def warpAffineRotation(imagePath, angle=0.0):
    ''' '''
    print(imagePath)
    image = cv2.imread(imagePath, cv2.IMREAD_COLOR)

    (h, w) = image.shape[:2]
    center = (w / 2, h / 2)
    angle = angle
    scale = 1

    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

imagePath='/home/jovyan/planets/BC503786-D500-4389-A1AC-2ABC8DA1D75F.jpeg'
img=cv2.imread(imagePath, cv2.IMREAD_COLOR)
img=cropSquare(img, 224, silent=False)
rotated=warpAffineRotation(imagePath, angle=67.5)
cvu.plotShowTwoImages(img, rotated)

img = img[:,:,::-1]
rotated = rotated[:,:,::-1]
fig, ax = plt.subplots(1, 2)
ax[0].imshow(img)
ax[1].imshow(rotated)

In [ ]:
def zoom_center(img, zoom_factor):
    ''' '''
    y_size = img.shape[0]
    x_size = img.shape[1]
    
    # define new boundaries
    x1 = int(0.5*x_size*(1-1/zoom_factor))
    x2 = int(x_size-0.5*x_size*(1-1/zoom_factor))
    y1 = int(0.5*y_size*(1-1/zoom_factor))
    y2 = int(y_size-0.5*y_size*(1-1/zoom_factor))

    # first crop image then scale
    img_cropped = img[y1:y2,x1:x2]
    cvu.plotShowTwoImages(img, img_cropped, title2='zoomed')
    return cv2.resize(img_cropped, None, fx=zoom_factor, fy=zoom_factor)

imagePath='/home/jovyan/planets/BC503786-D500-4389-A1AC-2ABC8DA1D75F.jpeg'
img=cv2.imread(imagePath, cv2.IMREAD_COLOR)
zm=zoom_center(img, zoom_factor=0.5)
cvu.plotShowTwoImages(img, zm)

img = img[:,:,::-1]
zm= zm[:,:,::-1]
fig, ax = plt.subplots(1, 2)
ax[0].imshow(img)
ax[1].imshow(zm)

In [ ]:
import numpy as np
from scipy.ndimage import zoom

def clippedZoom(img, zoom_factor, **kwargs):
    ''' '''
    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2
    return out
        
imagePath='/home/jovyan/planets/BC503786-D500-4389-A1AC-2ABC8DA1D75F.jpeg'
img=cv2.imread(imagePath, cv2.IMREAD_COLOR)
zm=clippedZoom(img, zoom_factor=0.75)
cvu.plotShowTwoImages(img, zm)

img = img[:,:,::-1]
zm= zm[:,:,::-1]
fig, ax = plt.subplots(1, 2)
ax[0].imshow(img)
ax[1].imshow(zm)

In [ ]:
def cv2_clipped_zoom(img, zoom_factor=0):
    """
    Center zoom in/out of the given image and returning an enlarged/shrinked view of 
    the image without changing dimensions
    ------
    Args:
        img : ndarray 
        Image array.
        zoom_factor : float
            amount of zoom as a ratio [0 to Inf). Default 0.
    ------
    Returns:
        result: ndarray
           numpy ndarray of the same shape of the input img zoomed 
           by the specified factor.          
    """
    if zoom_factor == 0:
        return img

    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
    
    ### Crop only the part that will remain in the result (more efficient)
    # Centered bbox of the final desired size in resized (larger/smaller) image coordinates
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]
    
    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)
    
    result = cv2.resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

In [ ]:
import numpy as np
from scipy.ndimage import zoom
def clipped_zoom(img, zoom_factor, **kwargs):
    ''' '''
    zero_pixel = img[0, 0]
    zero_pixel=float(zero_pixel[0])
    print(zero_pixel)
    print(type(zero_pixel))
    h, w = img.shape[:2]
    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:
        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2
        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)
    # Zooming in
    elif zoom_factor > 1:
        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [ ]:
def zoomInterpolation(image, zoom_factor):
    from scipy.ndimage.interpolation import zoom as izoom
    # print(type(izoom))
    zm_copy = np.copy(image)
    print('copy type:', type(zm_copy))

    zoom_image=izoom(zm_copy, zoom_factor, cval=255.0)
    zoom_image=cropSquare(zoom_image, 224)
    return(zoom_image)

imagePath='/home/jovyan/planets/BC503786-D500-4389-A1AC-2ABC8DA1D75F.jpeg'
img=cv2.imread(imagePath, cv2.IMREAD_COLOR)
# img=cropSquare(img, 224)
img=resize_maintain_aspect_ratio(imagePath)
# img = img[:,:,::-1]

zm1 = zoomInterpolation(img, 0.5)
# zm2 = zoomInterpolation(img, 1.1)

cvu.plotShowSingleImage(img, title1=img.shape)
cvu.plotShowSingleImage(zm1)
# cvu.plotShowSingleImage(zm2)
be='''
fig, ax = plt.subplots(1, 3)
ax[0].imshow(img)
ax[1].imshow(zm1)
ax[2].imshow(zm2)
'''

In [ ]:
#  remove initial crap
planetList=[]
os.chdir(planetsPath)
os.chdir(contentPath)
globList=glob.glob('**', recursive=False)
for fil in globList:
    if fil.endswith('jpeg')or fil.endswith('.png')or fil.endswith('.jpg'):
        fullPath=abspath(fil)
        planetList.append(fullPath)
        print(fullPath)
        
os.chdir(contentPath)
print(len(planetList), ' planets in planetList')
for path in planetList:
    os.remove(path)

In [ ]:
planetList=[]
os.chdir(planetsPath)
globList=glob.glob('**', recursive=True)
for fil in globList:
    if fil.endswith('jpeg')or fil.endswith('.png')or fil.endswith('.jpg'):
        fullPath=abspath(fil)
        planetList.append(fullPath)
        print(fullPath)
        
os.chdir(contentPath)
print(len(planetList), ' planets in planetList')

In [ ]:
import cv2
tempPathList=[]
planetCount=0
for pth in planetList:
    planetCount+=1
    planetName = 'planet' + str(planetCount) + '.png'
    savePath=join(contentPath, 'tmp')
    savePath=join(savePath, planetName)
    tempPathList.append(savePath)
    
    newImage = cv2.imread(pth, cv2.IMREAD_COLOR)
    newImage = resize_maintain_aspect_ratio(newImage, silent=False)
    
    if not exists(savePath):
        print(savePath)
        cv2.imwrite(filename=savePath, img=newImage)
        print(f'saved: {C.BIGreen}{savePath}{C.ColorOff}')
    else:
        print(f'{savePath}{C.BIPurple} already exists{C.ColorOff}')
    newImage=None

In [ ]:
# scale images
for path in tempPathList:
    # print(path)
    splt=splitext(path)
    # print(splt)
    savePath=splt[0] + 'z50'
    savePath=savePath + splt[1]
    print(savePath)
    
    img=cv2.imread(path, cv2.IMREAD_COLOR)
    zoomImg=np.copy(img)
    zoomImg=clipped_zoom(zoomImg, 0.5)
    cvu.plotShowTwoImages(img, zoomImg, title2=basename(savePath))
    
    img=None
    zoomImg=None

In [ ]:
q

In [ ]:
for path in tempPathList:
    splt=splitext(path)
    print(splt)
    
    
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    print(type(img))
    # cvu.plotShowSingleImage(img, title1=basename(path))
    

    img_flip_vert = cv2.flip(img, 0)
    cvu.plotShowTwoImages(img, img_flip_vert, title2='vertical')
    # cv2.imwrite('data/dst/lena_cv_flip_ud.jpg', img_flip_ud)
    # True

    img_flip_horiz = cv2.flip(img, 1)
    cvu.plotShowTwoImages(img, img_flip_horiz, title2='horizontal')
    # cv2.imwrite('data/dst/lena_cv_flip_lr.jpg', img_flip_lr)
    # True

    img_flip_both = cv2.flip(img, -1)
    cvu.plotShowTwoImages(img, img_flip_both, title2='both axis')
    # cv2.imwrite('data/dst/lena_cv_flip_ud_lr.jpg', img_flip_ud_lr)
    # True
    img=None
    img_flip_vert=None
    img_flip_horiz=None
    img_flip_both=None

In [ ]:
for path in tempPathList:
    rotatedImage = warpaffineRotation(imagePath=path, angle=22.5)
    cvu.plotShowSingleImage(rotatedImage, title1=basename(path))

In [ ]:
np.angle

In [ ]:
import numpy as np
from PIL import Image

img = np.array(Image.open('data/src/lena.jpg'))
print(type(img))
# <class 'numpy.ndarray'>

print(img.shape)
# (225, 400, 3)

Image.fromarray(np.rot90(img)).save('data/dst/lena_np_rot90.jpg')

Image.fromarray(np.rot90(img, 2)).save('data/dst/lena_np_rot90_180.jpg')

Image.fromarray(np.rot90(img, 3)).save('data/dst/lena_np_rot90_270.jpg')

In [ ]:
q
print('Original Dimensions : ',img.shape)

scale_percent = 200 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
 
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

print('Resized Dimensions : ', resized.shape)

cvu.plotShowTwoImages(img, resized, title2=resized.shape)

In [ ]:
def aspect_ratio_resize(image, width=224, height=None, inter=cv2.INTER_LINEAR_EXACT):
    # Grab the image size and initialize dimensions
    dim = None
    (h, w) = image.shape[:2]

    # Return original image if no need to resize
    if width is None and height is None:
        return image

    # We are resizing height if width is none
    if width is None:
        # Calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # We are resizing width if height is none
    else:
        # Calculate the ratio of the 0idth and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # Return the resized image
    resized=cv2.resize(image, dim, interpolation=inter)
    newSavePath=join(savePath, basename(imagePath))
    written=cv2.imwrite(newSavePath, resized)
    if written:
        return resized

In [ ]:
img=cv2.imread(newSavePath)
resizeImg=np.copy(img)
height = resizeImg.shape[0]
width = resizeImg.shape[1]
print(height, 'width:', width)

new_height = 224
ratio = new_height / height # (or new_height / height)
print(ratio)
new_height = int(height * ratio)
print(new_height)

dimensions = (width, new_height)
resizeImg = cv2.resize(img, (224, 224),interpolation=cv2.INTER_AREA)


cvu.plotShowTwoImages(img, resizeImg, title2=resizeImg.shape)